In [1]:
import pandas as pd
import os

database_path = "example/dataset"

beatmaps_csv = os.path.join(database_path, "beatmaps.csv")
hit_objects_csv = os.path.join(database_path, "hit_objects.csv")
time_points_csv = os.path.join(database_path, "timing_points.csv")

beatmaps_df = pd.read_csv(beatmaps_csv)
hit_objects_df = pd.read_csv(hit_objects_csv)
time_points_df = pd.read_csv(time_points_csv)


In [9]:
beatmaps_df.head()

,ID,Title,Artist,Creator,Version,HPDrainRate,CircleSize,OverallDifficulty,ApproachRate,SliderMultiplier,SliderTickRate
0,2269930-0,Favorite Liar (Cut Ver.),The Wrecks,PikAqours,Kuroame's Favorite Liar,5.5,3.8,9.6,9.7,1.80,1.0
1,2269930-1,Favorite Liar (Cut Ver.),The Wrecks,PikAqours,Harumi's Normal,3.0,3.2,3.8,5.8,0.90,1.0
2,2269930-2,Favorite Liar (Cut Ver.),The Wrecks,PikAqours,Amats' Insane,5.0,3.6,8.0,9.0,1.55,1.0
3,2269930-3,Favorite Liar (Cut Ver.),The Wrecks,PikAqours,Kuki's Extra,5.0,3.8,9.2,9.4,1.80,1.0
4,2269930-4,Favorite Liar (Cut Ver.),The Wrecks,PikAqours,Parad0xa's Expert,5.2,3.6,8.8,9.2,1.60,0.5


In [10]:
hit_objects_df.head()

,ID,Time,Type,X,Y,HitSound,Extra
0,2269930-0,8262,slider,367,107,0,L|373:36
1,2269930-0,8643,slider,428,184,0,P|364:152|278:184
2,2269930-0,9021,slider,261,268,0,L|340:262
3,2269930-0,9273,slider,300,364,0,L|144:353
4,2269930-0,9783,slider,12,261,0,L|168:272


In [11]:
time_points_df.head()

,ID,offset,ms_per_beat,time_signature,meter,sample_set,sample_index,volume,effects
0,2269930-0,245,266.666667,4,2,1,60,1,0
1,2269930-0,645,247.933884,4,2,1,60,1,0
2,2269930-0,1264,263.157895,4,2,1,60,1,0
3,2269930-0,1790,244.897959,4,2,1,60,1,0
4,2269930-0,2279,241.935484,4,2,1,60,1,0
